<a href="https://colab.research.google.com/github/LeoFernanndes/notebooks/blob/master/Sonar_Classification_with_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sonar Dataset Classification with Keras

In [2]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


data_url = 'https://raw.githubusercontent.com/LeoFernanndes/datasets/master/sonar.all-data.csv'
dataframe = pd.read_csv(data_url)
dataset = dataframe.values

x = dataset[:, 0:60].astype(float)
y = dataset[:, 60]

Using TensorFlow backend.


In [0]:
# encoding categorical feature
encoder = LabelEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)

### Baseline Sklearn Model

In [23]:
from xgboost import XGBClassifier

ss = StandardScaler()
x_scaled = ss.fit_transform(x)

xgb = XGBClassifier()
kfold = StratifiedKFold(n_splits= 10, shuffle= True, random_state= 42)
results = cross_val_score(xgb, x_scaled, encoded_y, cv= kfold)
print('Baseline Sklearn accuracy: {}% ± {}%'.format(100 * results.mean(), 200 * results.std()))

Baseline Sklearn accuracy: 85.49999999999999% ± 11.612507917588449%


### Baseline Model

In [0]:
# baseline model
def create_baseline():
  # create model
  model = Sequential()
  model.add(Dense(60, input_dim= 60, activation= 'relu'))
  model.add(Dense(1, activation= 'sigmoid'))
  # compile model
  model.compile(loss= 'binary_crossentropy', optimizer= 'adam', metrics= ['accuracy'])
  return model

In [6]:
# evaluate odel with standardized dataset
estimator = KerasClassifier(build_fn= create_baseline, epochs= 100, batch_size= 5, verbose= 0)
kfold = StratifiedKFold(n_splits= 10, shuffle= True)
results = cross_val_score(estimator, x, encoded_y, cv= kfold)
print('Baseline: {:2.2f}% ± {:2.2f}%'.format(100 * results.mean(), 200 * results.std()))

Baseline: 81.24% ± 13.75%


In [7]:
# pipepine
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn= create_baseline, epochs= 100, batch_size= 5, verbose= 0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits= 10, shuffle= True)
results = cross_val_score(pipeline, x, encoded_y, cv= kfold)
print('Standardized: {:2.2f}% ± {:2.2f}%'.format(100 * results.mean(), 200 * results.std()))

Standardized: 86.50% ± 20.02%


### Smaller Model

In [0]:
# smaller model
def create_smaller():
  # create model
  model = Sequential()
  model.add(Dense(30, input_dim= 60, activation= 'relu'))
  model.add(Dense(1, activation= 'sigmoid'))
  # compile model
  model.compile(loss= 'binary_crossentropy', optimizer= 'adam', metrics= ['accuracy'])
  return model

In [9]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn= create_smaller, epochs= 100, batch_size= 5, verbose= 0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits= 10, shuffle= True)
results = cross_val_score(pipeline, x, encoded_y, cv= kfold)
print('Smaller: {:2.2f}% ± {:2.2f}%'.format(100 * results.mean(), 200 * results.std()))

Smaller: 84.48% ± 14.40%


### Larger Model

In [0]:
# larger model
def create_larger():
  # create model
  model = Sequential()
  model.add(Dense(60, input_dim= 60, activation= 'relu'))
  model.add(Dense(30, activation= 'relu'))
  model.add(Dense(1, activation= 'sigmoid'))
  # compile model
  model.compile(loss= 'binary_crossentropy', optimizer= 'adam', metrics= ['accuracy'])
  return model

In [11]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn= create_larger, epochs= 100, batch_size= 5, verbose= 0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits= 10, shuffle= True)
results = cross_val_score(pipeline, x, encoded_y, cv= kfold)
print('Larger: {:2.2f}% ± {:2.2f}%'.format(100 * results.mean(), 200 * results.std()))

Larger: 87.48% ± 12.15%
